In [2]:
import pandas as pd
df = pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
x = df.drop(labels=['price'],axis=1)
y = df[['price']]

In [4]:
cat=x.select_dtypes(include="object").columns
num=x.select_dtypes(exclude="object").columns

In [5]:
cat

Index(['cut', 'color', 'clarity'], dtype='object')

In [6]:
num

Index(['id', 'carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [7]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [8]:
# auto mating the process therefore pipelining
from sklearn.impute import SimpleImputer #handling missing value
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OrdinalEncoder

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
# numerical pipeline
num_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="median")),
        ("scaler",StandardScaler())
    ]
)

In [12]:
cat_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),
        ("ordinalencoder",OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ("scaler",StandardScaler())
    ]
)

In [13]:
preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,num),
('cat_pipeline',cat_pipeline,cat)
])

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=30)

In [16]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

# remember test data o sirf transform and training ko fit_transform

In [17]:
x_train.head()

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,1.274919,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,-1.102907,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,-0.997613,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.173764,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.914812,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [18]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [19]:

regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [20]:
regression.coef_

array([[-4.49143681e-02,  6.43365918e+03, -1.32758240e+02,
        -7.04288742e+01, -1.72031317e+03, -4.99289729e+02,
        -6.33921992e+01,  7.24455152e+01, -4.60416107e+02,
         6.50764277e+02]])

In [21]:

regression.intercept_

array([3970.76628955])

In [22]:

import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [30]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    
    y_pred=model.predict(x_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1013.904802565844
MAE: 674.0257224619531
R2 score 93.68908132630604


Lasso
Model Training Performance
RMSE: 1013.8784227049385
MAE: 675.071691842108
R2 score 93.68940971806553


Ridge
Model Training Performance
RMSE: 1013.9060209095863
MAE: 674.0557936094749
R2 score 93.68906615945235


Elasticnet
Model Training Performance
RMSE: 1533.4194703386765
MAE: 1060.742542700526
R2 score 85.56488759855306




In [31]:
model_list


['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [32]:
trained_model_list

[]